In [ ]:
import pymysql
import datetime
import json
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv

In [ ]:
# DB연결
load_dotenv()

conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    db=os.getenv("DB_NAME"),
    charset=os.getenv("DB_CHARSET", "utf8mb4")
    )

In [ ]:
# 서비스 이용하면 이용 기록 바로 가져오기
nickname = "EUNJI" # 사용자 닉네임 EUNJI는 예시

cur = conn.cursor(pymysql.cursors.DictCursor)
# 날짜로 내림차순을 해서 첫번째만 가져오기 = 제일 최신 사용 정보 가져오기
cur.execute("""
    SELECT e.nickname, depression, anxiety, stress, happiness, achievement, 
           energy, category_1, category_2, category_3, park_1, park_2, park_3
    FROM tb_users_emotions e 
    JOIN tb_users_category_recommend u ON u.id = e.id
    JOIN tb_users_parks_recommend p ON e.id = p.id
    WHERE e.nickname = %s
    ORDER BY createdate DESC
    LIMIT 1;
""", (nickname, )) 
use = cur.fetchall()


cur.close()
conn.close()

In [ ]:
# 한 번 사용당 요약
summary_prompt = PromptTemplate.from_template("""
Input Data:
우울: {depression}
불안: {anxiety}
스트레스: {stress}
행복: {happiness}
성취감: {achievement}
에너지: {energy}
category_1: {category_1}
category_2: {category_2}
category_3: {category_3}
park_1: {park_1}
park_2: {park_2}
park_3: {park_3}

Instructions:
1. 감정 top3를 점수 순으로 내림차순 정렬해서 "top_emotions"에 담아주세요.
   - 점수가 같으면 모두 포함할 수 있음 (즉, top3 이상이 될 수도 있음)
2. 각 감정은 {{"감정명": 점수}} 형태로 작성
   - 예: [{{"우울": 3}}, {{"불안": 2}}, {{"행복": 5}}]
3. 전체 감정의 흐름을 자연어로 요약해서 "emotions_summary"에 한 줄(50자 이내)로 담아주세요.
4. 추천 카테고리(category_1~3)를 **각각 짧게 요약**해서 리스트 형태로 "recommand_cates"에 담아주세요.
5. 추천 공원(park_1~3)을 그대로(**절대 이름을 바꾸지 말고**) 리스트 형태로 "recommand_parks"에 담아주세요.
6. JSON 형식을 정확히 지켜주세요.

Return JSON in this exact format:
{{
  "top_emotions": [],
  "emotions_summary": "",
  "recommand_cates": [],
  "recommand_parks": []
}}
""")

summary_chain = summary_prompt | ChatOpenAI(model="gpt-4o-mini") | JsonOutputParser()

summary = summary_chain.invoke(use)
print(summary)

In [ ]:
with conn.cursor() as cur:
    sql = """
    INSERT INTO tb_emotion_summary
    (nickname, Create_date, TopEmotions, EmotionsSummary, RecommandCates, RecommandParks)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cur.execute(sql, (
        summary['nickname'],
        summary['Create_date'].strftime('%Y-%m-%d %H:%M:%S'),  # DATETIME 형식
        json.dumps(summary['top_emotions'], ensure_ascii=False),
        summary['emotions_summary'],
        json.dumps(summary['recommand_cates'], ensure_ascii=False),
        json.dumps(summary['recommand_parks'], ensure_ascii=False)
    ))

    conn.commit()

conn.close()

In [ ]:
# 전 주 날짜 가져오기(datetime 맞춰서 시간까지 가져오기)
# 파이썬 일주일은 월요일이 한 주의 시작
def get_last_week_range():
    today = datetime.date.today()
    
    # 이번 주 월요일 (00:00:00 기준)
    this_week_monday = today - datetime.timedelta(days=today.weekday())
    
    # 지난 주 월요일과 일요일
    last_week_monday = this_week_monday - datetime.timedelta(days=7)
    last_week_sunday = this_week_monday - datetime.timedelta(days=1)
    
    # DATETIME 범위로 변환
    start_dt = datetime.datetime.combine(last_week_monday, datetime.time.min)  # 00:00:00
    end_dt   = datetime.datetime.combine(last_week_sunday, datetime.time.max)  # 23:59:59.999999
    
    return start_dt, end_dt

start_dt, end_dt = get_last_week_range()

In [ ]:
# 사용 요약본 일주일치 가져오기
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("""
    SELECT *
    FROM tb_emotion_summary  
    WHERE nickname = %s
      AND Create_date BETWEEN %s AND %s
    ;""", (nickname, start_dt, end_dt))
week_list = cur.fetchall()

cur.close()
conn.close()

# 한 주에 사용량이 3번이 안되면
if len(week_list) < 3:
    print('요약할 데이터가 충분하지 않습니다.')
else:
    pass

# 일주일치 요약본 문자열 하나로 만들기
conctents = "\n\n".join(
    [
        f"Record {i+1} ({day['Create_date'].strftime('%m/%d %H:%M') if 'Create_date' in day else ''})\n"
        f"감정_top3: {day['TopEmotions']}\n"
        f"감정_요약: {day['EmotionsSummary']}\n"
        f"추천_카테고리: {day['RecommandCates']}\n"
        f"추천_공원: {day['RecommandParks']}"
        for i, day in enumerate(week_list)
    ]
)

# 사용자 이름, 일주일치 요약본 합쳐서 llm 넣을 재료 만들기
weekly_text = {}
weekly_text['nickname'] = week_list[0]['nickname']
weekly_text['conctents'] = conctents

In [ ]:
# 최종 평가 llm
overall_prompt = PromptTemplate.from_template("""
# Guidelines
- Use only the information provided.
- Do not make up information.
- Do not exaggerate.

당신은 따뜻한 위로의 말을 전해주는 상담사입니다. 
아래는 한 사용자의 한 주 동안 서비스 이용 결과야. 
데이터 그대로 말하지 말고, 약간의 관찰과 해석, 따뜻한 격려를 담아서 총평을 작성해줘. 
말투는 사무적이지 않고 자연스럽게, 상담사가 말하듯 부드럽게 작성해.
**"사용자"라는 단어는 사용하지 마세요. 대신 UserNickname님이라고 한 번만 언급하세요.**

총평은 5줄 내외, 자연스러운 완전 문장으로 작성해주세요.

Inputs
UserNickname: {nickname}
주간 데이터: {conctents}

Return in JSON format:
"nickname": {nickname}
"review": ""
""")

overall_chain = overall_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.5) | JsonOutputParser()
weekly_review = overall_chain.invoke(weekly_text)

In [ ]:
# 총평 저장 (create_date는 총평을 만드는 날짜로 인서트)
with conn.cursor() as cur:
    sql = """
    INSERT INTO tb_weekly_review (nickname, create_date, review)
    VALUES (%s, %s, %s)
    """
    cur.execute(sql, (weekly_review['nickname'], datetime.now(), weekly_review['review']))
    conn.commit()

conn.close()